# Algoritmo de predicción

In [1]:
# IMPORTAR LIBRERIAS
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import ElasticNet
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm


In [2]:
# CARGA DE DATOS
url_cliente = "https://raw.githubusercontent.com/annaalfaro/EntregaTFM/main/Cliente.csv"
url_producto = "https://raw.githubusercontent.com/annaalfaro/EntregaTFM/main/Producto.csv"
url_ventas   = "https://raw.githubusercontent.com/annaalfaro/EntregaTFM/main/Ventas_2014-2024.csv"

Cliente = pd.read_csv(url_cliente)
Producto = pd.read_csv(url_producto)
ventas = pd.read_csv(url_ventas)

In [3]:
# Procesado de datos
ventas['fecha'] = pd.to_datetime(ventas['fecha'])
ventas['año'] = ventas['fecha'].dt.year
ventas['mes'] = ventas['fecha'].dt.month

# AgrupaR por cliente, producto, año y mes
ventas_grouped = ventas.groupby(['códigocliente', 'códigoproducto', 'año', 'mes'])['unidades'].sum().reset_index()

# Codificar cliente y producto
ventas_grouped['cliente_id'] = ventas_grouped['códigocliente'].astype('category').cat.codes
ventas_grouped['producto_id'] = ventas_grouped['códigoproducto'].astype('category').cat.codes

In [4]:
# Crear datos de entrenamiento con variables adicionales (lags y medias móviles)

# Crear columna de unidades anteriores (lag_12: mismo mes año anterior)
ventas_grouped = ventas_grouped.sort_values(by=['cliente_id', 'producto_id', 'año', 'mes'])
ventas_grouped['lag_12'] = ventas_grouped.groupby(['cliente_id', 'producto_id'])['unidades'].shift(1)

# Crear media de últimos 3 valores históricos
ventas_grouped['mean_last_3'] = ventas_grouped.groupby(['cliente_id', 'producto_id'])['unidades'].transform(lambda x: x.shift(1).rolling(3).mean())

# Crear tendencia: diferencia con año anterior
ventas_grouped['unidades_prev'] = ventas_grouped.groupby(['cliente_id', 'producto_id'])['unidades'].shift(1)
ventas_grouped['trend'] = ventas_grouped['unidades'] - ventas_grouped['unidades_prev']

# Eliminar filas con NaNs en nuevas variables
ventas_grouped = ventas_grouped.dropna(subset=['lag_12', 'mean_last_3', 'trend'])
features = ['año', 'mes', 'cliente_id', 'producto_id', 'lag_12', 'mean_last_3', 'trend']
target = 'unidades'

# Preparar datos de test y de entreno
train_data = ventas_grouped[ventas_grouped['año'] < 2024]
test_data = ventas_grouped[ventas_grouped['año'] == 2024]

X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]


In [5]:
# Modelos predictivos: XGBoost, LightGBM, CatBoost, ElasticNet
modelo_xgb = XGBRegressor(random_state=42)
modelo_lgbm = LGBMRegressor(random_state=42)
modelo_catboost = CatBoostRegressor(verbose=0, random_state=42)
modelo_enet = ElasticNet(random_state=42)
modelo = XGBRegressor(random_state=42)
modelo.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [6]:
# Entrenar cada modelo
modelo_xgb.fit(X_train, y_train)
modelo_lgbm.fit(X_train, y_train)
modelo_catboost.fit(X_train, y_train)
modelo_enet.fit(X_train, y_train)


ElasticNet(random_state=42)

In [7]:
# Predicciones
pred_xgb = modelo_xgb.predict(X_test)
pred_lgbm = modelo_lgbm.predict(X_test)
pred_catboost = modelo_catboost.predict(X_test)
pred_enet = modelo_enet.predict(X_test)

# Promedio de predicciones
y_pred = (pred_xgb + pred_lgbm + pred_catboost + pred_enet) / 4


In [8]:
# Evaluación por cliente + producto + mes con métricas por rangos

def evaluar_por_rangos(df, y_true_col, y_pred_col):
    rangos = [(0, 5), (5, 10), (10, 20), (20, 50), (50, np.inf)]
    print("Evaluación por rangos de unidades reales (cliente-producto-mes):")
    for r_min, r_max in rangos:
        subset = df[(df[y_true_col] > r_min) & (df[y_true_col] <= r_max)]
        if not subset.empty:
            mse = mean_squared_error(subset[y_true_col], subset[y_pred_col])
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(subset[y_true_col], subset[y_pred_col])
            r2 = r2_score(subset[y_true_col], subset[y_pred_col])
            mape = np.mean(np.abs((subset[y_true_col] - subset[y_pred_col]) / np.maximum(subset[y_true_col], 1))) * 100
            print(f"Rango ({r_min}, {r_max}]: MSE={mse:.2f}, RMSE={rmse:.2f}, MAE={mae:.2f}, R2={r2:.2f}, MAPE={mape:.2f}%")
        else:
            print(f"Rango ({r_min}, {r_max}]: Sin datos")

# Comparar predicción con test_data
pred_df_eval = test_data.copy()
pred_df_eval['y_pred'] = y_pred
evaluar_por_rangos(pred_df_eval, 'unidades', 'y_pred')


Evaluación por rangos de unidades reales (cliente-producto-mes):
Rango (0, 5]: MSE=0.01, RMSE=0.07, MAE=0.06, R2=1.00, MAPE=3.70%
Rango (5, 10]: MSE=0.01, RMSE=0.08, MAE=0.05, R2=1.00, MAPE=0.61%
Rango (10, 20]: MSE=0.03, RMSE=0.17, MAE=0.13, R2=1.00, MAPE=0.86%
Rango (20, 50]: MSE=0.20, RMSE=0.45, MAE=0.33, R2=0.99, MAPE=1.24%
Rango (50, inf]: Sin datos


In [9]:
# Generar predicciones para 2025 usando ensamblado de modelos
clientes = ventas_grouped['cliente_id'].unique()
productos = ventas_grouped['producto_id'].unique()
meses = list(range(1, 13))
años_pred = [2025]
predicciones = []

for cliente in clientes:
    for producto in productos:
        for mes in meses:
            fila_historica = ventas_grouped[
                (ventas_grouped['cliente_id'] == cliente) &
                (ventas_grouped['producto_id'] == producto)
            ]

            fila_lag = fila_historica[
                (fila_historica['año'] == 2024) &
                (fila_historica['mes'] == mes)
            ]
            lag_12 = fila_lag['unidades'].values[0] if not fila_lag.empty else np.nan

            mean_last_3 = fila_historica[
                (fila_historica['mes'] == mes) &
                (fila_historica['año'] >= 2021) &
                (fila_historica['año'] <= 2023)
            ]['unidades'].mean()

            fila_pred = pd.DataFrame({
                'año': años_pred,
                'mes': [mes],
                'cliente_id': [cliente],
                'producto_id': [producto],
                'lag_12': [lag_12],
                'mean_last_3': [mean_last_3],
                'trend': [0]
            }).dropna()

            if not fila_pred.empty:
                pred_ensamble = (
                    modelo_xgb.predict(fila_pred)[0] +
                    modelo_lgbm.predict(fila_pred)[0] +
                    modelo_catboost.predict(fila_pred)[0] +
                    modelo_enet.predict(fila_pred)[0]
                ) / 4

                pred_ensamble = np.clip(pred_ensamble, 0, np.percentile(y_train, 99))
                predicciones.append([cliente, producto, 2025, mes, pred_ensamble])

# Crear DataFrame final
pred_df = pd.DataFrame(predicciones, columns=['cliente_id', 'producto_id', 'año', 'mes', 'unidades_predichas'])



In [10]:
# Reconvertir códigos a etiquetas originales
cliente_map = dict(enumerate(ventas_grouped['códigocliente'].astype('category').cat.categories))
producto_map = dict(enumerate(ventas_grouped['códigoproducto'].astype('category').cat.categories))
pred_df['códigocliente'] = pred_df['cliente_id'].map(cliente_map)
pred_df['códigoproducto'] = pred_df['producto_id'].map(producto_map)
pred_df['fecha'] = pd.to_datetime(dict(year=pred_df['año'], month=pred_df['mes'], day=1))

In [11]:
# Preparar datos para generar el csv.
pred_df['fecha'] = pd.to_datetime(dict(year=pred_df['año'], month=pred_df['mes'], day=1))
pred_df['fecha'] += pd.to_timedelta(np.random.randint(0, 28, size=len(pred_df)), unit='D')

# Reconvertir códigos a etiquetas originales
cliente_map = dict(enumerate(ventas_grouped['códigocliente'].astype('category').cat.categories))
producto_map = dict(enumerate(ventas_grouped['códigoproducto'].astype('category').cat.categories))
pred_df['códigocliente'] = pred_df['cliente_id'].map(cliente_map)
pred_df['códigoproducto'] = pred_df['producto_id'].map(producto_map)
pred_df['unidades_predichas'] = np.round(pred_df['unidades_predichas']).astype(int)

# Añadir precio total = unidades_predichas * precio producto
Producto['precio'] = Producto['precio de venta (€)'].astype(str).str.replace(',', '.').astype(float)
precio_map = dict(zip(Producto['códigoproducto'], Producto['precio']))
pred_df['preciototal'] = pred_df.apply(
    lambda row: row['unidades_predichas'] * precio_map.get(row['códigoproducto'], 0), axis=1
)

# Crear código de venta incremental basado en el último valor real
ventas['códigoventas'] = ventas['códigoventas'].astype(str)
ultimo_codigo_real = ventas['códigoventas'].dropna().sort_values().iloc[-1]
numero_base = int(ultimo_codigo_real.strip('V').lstrip('0')) + 1

# Formatear y ordenar predicciones
pred_df_sorted = pred_df.sort_values(by='fecha').copy()
pred_df_sorted.reset_index(drop=True, inplace=True)
pred_df_sorted['código_venta'] = ['V' + str(numero_base + i).zfill(9) for i in range(len(pred_df_sorted))]

# Seleccionar final de columnas
columnas_finales = ['código_venta', 'fecha', 'códigocliente', 'códigoproducto', 'unidades_predichas', 'preciototal']
pred_df_sorted = pred_df_sorted[columnas_finales]

# Exportar a CSV
pred_df_sorted.to_csv("predicciones_2025.csv", index=False)
